In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")

In [3]:
train_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [4]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)

In [5]:
train_df

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D
...,...,...,...,...,...,...,...,...
195,195,What is the relation between the three moment ...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,C
196,196,"What is the throttling process, and why is it ...",The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,B
197,197,What happens to excess base metal as a solutio...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",B
198,198,"What is the relationship between mass, force, ...",Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,D


In [6]:
from transformers import AutoTokenizer

model_dir = "/kaggle/input/huggingface-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [7]:
options = "ABCDE"
indices = list(range(5)) 

option_to_index = {option:index for option,index in zip(options,indices)}
index_to_options = {index:option for option,index in zip(options,indices)}


attention mask:It's a binary mask that helps the model focus on relevant tokens while ignoring padding tokens during training and inference.

In [8]:
def preprocess(example):
    first_sentece = [example["prompt"]]*5
    second_sentece = []
    for option in options:
        second_sentece.append(example[option])
    tokenzied_example = tokenizer(first_sentece,second_sentece,truncation=True)
    tokenzied_example["label"] = option_to_index[example["answer"]]
    return tokenzied_example

In [9]:
train_ds

Dataset({
    features: ['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 200
})

In [10]:
tokened_train_ds = train_ds.map(preprocess,batched=False,remove_columns=["prompt","A","B","C","D","E","answer"])

  0%|          | 0/200 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase,PaddingStrategy
from typing import Optional, Union
import torch
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer : PreTrainedTokenizerBase
    padding : Union[bool,str,PaddingStrategy] = True
    max_length : Optional[int] = None
    pad_to_multiple_of : Optional[int] = None
    
    def __call__(self,features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_chocies = len(features[0]["input_ids"])
        flattend_features = [[{k:v[i] for k,v in feature.items()} 
                              for i in range(num_chocies)] for feature in features]
        falltened_features = sum(flattend_features,[])
        batch = self.tokenizer.pad(falltened_features,padding=self.padding,max_length=self.max_length
                                  ,pad_to_multiple_of=self.pad_to_multiple_of,return_tensors="pt")
        batch = {k:v.view(batch_size,num_chocies,-1) for k,v in batch.items()}
        batch["labels"] = torch.tensor(labels,dtype=torch.int64)
        return batch

In [12]:
from transformers import AutoModelForMultipleChoice,TrainingArguments,Trainer
model = AutoModelForMultipleChoice.from_pretrained(model_dir)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [13]:
dirt = "fine_tunnet_bert"
training_args = TrainingArguments(output_dir=dirt,evaluation_strategy="epoch",save_strategy="epoch",load_best_model_at_end=True,
                                learning_rate=5e-5,per_device_train_batch_size=4,per_device_eval_batch_size=4,num_train_epochs=4,
                                weight_decay=0.01,report_to="none")

In [14]:
trainer = Trainer(model=model,args=training_args,train_dataset=tokened_train_ds,
                 eval_dataset=tokened_train_ds,tokenizer=tokenizer,
                 data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer))

In [15]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.460087
2,No log,1.263466
3,No log,1.179439
4,No log,0.881609


TrainOutput(global_step=200, training_loss=1.368470458984375, metrics={'train_runtime': 58.5743, 'train_samples_per_second': 13.658, 'train_steps_per_second': 3.414, 'total_flos': 180784449598800.0, 'train_loss': 1.368470458984375, 'epoch': 4.0})

In [16]:
predications = trainer.predict(tokened_train_ds)

In [17]:
tokened_train_ds

Dataset({
    features: ['id', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 200
})

In [27]:
import numpy as np
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_to_options.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [28]:
predictions_to_map_output(predications.predictions)

array(['D B E', 'A B D', 'A C E', 'A C D', 'D E C', 'B C E', 'A C D',
       'D E B', 'C E D', 'A C E', 'A E D', 'A D B', 'C B E', 'D E C',
       'D C E', 'B E C', 'E B C', 'E A B', 'A B D', 'E C D', 'B D C',
       'E D B', 'C B A', 'C A B', 'E A D', 'E B D', 'A C E', 'D C B',
       'E B C', 'C B A', 'B D E', 'E A C', 'E B D', 'D A E', 'C E A',
       'B D E', 'E A D', 'A D C', 'E D A', 'E A B', 'E D A', 'D E C',
       'B C D', 'D C B', 'A B D', 'A B C', 'B E C', 'C B A', 'E D B',
       'B A E', 'B D E', 'E B D', 'C A B', 'D A C', 'B A C', 'B E D',
       'C A D', 'A C B', 'D A B', 'B C A', 'B D E', 'E D B', 'C A B',
       'C B A', 'A D E', 'E A C', 'C D E', 'E C A', 'C E D', 'D E C',
       'B C A', 'E A D', 'D E A', 'B E D', 'D B C', 'B E D', 'B A D',
       'B A D', 'C B A', 'E A C', 'C D E', 'A C D', 'B D C', 'A E D',
       'C A D', 'B D A', 'D B A', 'B D A', 'A E C', 'B D C', 'D B A',
       'B D C', 'E B C', 'E D A', 'E B D', 'C D A', 'C A D', 'D B E',
       'C D E', 'D E

In [20]:
test_df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")

In [21]:
test_df.head()

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [22]:
test_df['answer'] = 'A'

In [23]:
test_ds = Dataset.from_pandas(test_df)
tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

  0%|          | 0/200 [00:00<?, ?ex/s]

In [24]:
test_predications = trainer.predict(tokenized_test_ds)